In [2]:
# !pip install apache-beam[gcp]

In [63]:
import os
import tensorflow as tf 
import apache_beam as beam 
import tensorflow_data_validation as tfdv
from google.cloud import bigquery
from datetime import datetime
import numpy as np

# from tensorflow_transform.tf_metadata import dataset_metadata
# from tensorflow_transform.tf_metadata import dataset_schema

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

print("TF version:", tf.__version__)
print("TFDV version:", tfdv.__version__)
print("Beam version:", beam.__version__)
print("BQ SDK version:", bigquery.__version__)

TF version: 2.9.0-rc2
TFDV version: 1.9.0
Beam version: 2.40.0
BQ SDK version: 2.34.4


## Helpful Guides
* [tfdv-structured-data-flights](https://notebook.community/GoogleCloudPlatform/mlops-on-gcp/examples/tfdv-structured-data/tfdv-flights)
* [sequential-data from BQ to TF-Records with apache beam](https://blog.ivanukhov.com/2019/11/08/sequential-data.html)
* [forecast-pipeline - from BQ to TF-Records with apache beam](https://github.com/chain-rule/example-weather-forecast/blob/master/forecast/pipeline.py)
* [Dataflow - Create TF-Records](https://quantiphi.com/scaling-etl-pipeline-for-creating-tensorflow-records-using-apache-beam-python-sdk-on-google-cloud-dataflow/)

# Apache Beam Pipeline -> write tf-records from BQ

### Table rows
* BigQueryIO read and write transforms produce and consume data as a `PCollection` of dictionaries, where each element in the `PCollection` represents a single row in the table

### Schemas
* When writing to BigQuery, you must supply a table schema for the destination table that you want to write to, unless you specify a create disposition of CREATE_NEVER. 
* Creating a table schema covers schemas in more detail.

In [8]:
PROJECT_ID = 'hybrid-vertex'
BUCKET_NAME = 'spotify-beam-v1' # 'spotify-tfrecords-blog' # Set your Bucket name
REGION = 'us-central1' # Set the region for Dataflow jobs
VERSION = 'v1'

ROOT = f'gs://{BUCKET_NAME}/{VERSION}'

CANDIDATE_DIR = ROOT + "/candidates/"
DATA_DIR = ROOT + '/data/' # Location to store data
STATS_DIR = ROOT +'/stats/' # Location to store stats 
STAGING_DIR = ROOT + '/job/staging/' # Dataflow staging directory on GCP
TEMP_DIR =  ROOT + '/job/temp/' # Dataflow temporary directory on GCP

### Cleanup working directory

In [18]:
# test_root ='gs://spotify-tfrecords-blog/candidate_tracks_v2'
# files = np.array(tf.io.gfile.listdir(test_root))
# for file in files:
#     file_to_remove = f'{test_root}/{file}'
#     print(file_to_remove)
#     break
#     # tf.io.gfile.remove(f'{test_root}/{file})
# # files

In [19]:
if tf.io.gfile.exists(ROOT):
    print("Removing {} contents...".format(ROOT))
    files = np.array(tf.io.gfile.listdir(ROOT))
    for file in files:
        tf.io.gfile.remove(f'{ROOT}/{file}')

print("Creating working directory: {}".format(ROOT))
tf.io.gfile.mkdir(ROOT)

Creating working directory: gs://spotify-beam-v1/v1


## Extract data from BigQuery to GCS

* extract data from BigQuery, 
* convert data to TFRecord files, 
* store data files in Google Cloud Storage (GCS). 
> * This data file in GCS will then be used by TFDV for stats & vocabs

### Define SQL Query

In [20]:
def generate_query(project=None, bq_dataset=None, bq_table=None, limit=None):
    query =f"""
        SELECT *
        FROM 
          `{project}.{bq_dataset}.{bq_table}`
        """
    if limit:
        query  += "LIMIT {}".format(limit)
        
    return query

In [21]:
from tensorflow.train import BytesList, Feature, FeatureList, Int64List, FloatList
from tensorflow.train import SequenceExample, FeatureLists


def string_array(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v.encode('utf-8') for v in value]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v) for v in value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[int(v) for v in value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def get_type_map(query):
    bq_client = bigquery.Client()
    query_job = bq_client.query(f"{query}")
    results = query_job.result()
    
    type_map = {}
    
    for field in results.schema:
        type_map[field.name] = field.field_type
    
    return type_map

# def get_mode_map(query):
#     bq_client = bigquery.Client()
#     query_job = bq_client.query(f"{query}")
#     results = query_job.result()
    
#     mode_map = {}
    
#     for field in results.schema:
#         mode_map[field.name] = field.mode
    
#     return mode_map


def row_to_example(instance, type_map):
    feature = {}
    for key, value in instance.items():
        data_type = type_map[key]
        if value is None:
            feature[key] = tf.train.Feature()
        elif data_type == 'INTEGER':
            feature[key] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=[value]))
        elif data_type == 'FLOAT':
            feature[key] = tf.train.Feature(
                float_list=tf.train.FloatList(value=[value]))
        else:
            feature[key] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(value)]))
            
    return tf.train.Example(features=tf.train.Features(feature=feature))

## Submit Pipeline

In [22]:
def run_pipeline(args):

    source_query = args.pop('source_query')
    sink_data_location = args.pop('sink_data_location')
    runner = args['runner']
    
    pipeline_options = beam.options.pipeline_options.GoogleCloudOptions(**args)
    print(pipeline_options)
    
    with beam.Pipeline(runner, options=pipeline_options) as pipeline:
        (pipeline 
         | "Read from BigQuery">> beam.io.Read(beam.io.BigQuerySource(query = source_query, use_standard_sql = True))
         | 'Convert to tf Example' >> beam.Map(lambda instance: row_to_example(instance, type_map))
         | 'Serialize to String' >> beam.Map(lambda example: example.SerializeToString(deterministic=True))
         | "Write as TFRecords to GCS" >> beam.io.WriteToTFRecord(
                    file_path_prefix = sink_data_location+"candidates", 
                    file_name_suffix=".tfrecord")
        )

In [23]:
runner = 'DataflowRunner'
job_name = 'spotify-tfrecord-beam-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S'))


print("Generating source query...")
LIMIT = 10000
PROJECT_ID='hybrid-vertex'
BQ_DATASET='spotify_train_4'
BQ_TABLE = 'candidate_features_v2'

data_location = CANDIDATE_DIR

source_query = generate_query(project=PROJECT_ID, bq_dataset=BQ_DATASET, bq_table=BQ_TABLE, limit=LIMIT)

print("Retrieving data type...")
type_map = get_type_map(source_query)

args = {
    'job_name': job_name,
    'runner': runner,
    'source_query': source_query,
    'type_map': type_map,
    'sink_data_location': data_location,
    'project': PROJECT_ID,
    'region': REGION,
    'staging_location': STAGING_DIR,
    'temp_location': TEMP_DIR,
    # 'save_main_session': True,
    # 'setup_file': './setup.py'
}
print("Pipeline args are set.")

Generating source query...
Retrieving data type...
Pipeline args are set.


In [25]:
# tf.logging.set_verbosity(tf.logging.ERROR)

print("Running data extraction pipeline...")
run_pipeline(args)
print("Pipeline is done.")

Running data extraction pipeline...
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, impersonate_service_account=None, job_name=spotify-tfrecord-beam-220711-142630, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://spotify-beam-v1/v1/job/staging/, temp_location=gs://spotify-beam-v1/v1/job/temp/, template_location=None, transform_name_mapping=None, update=False)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: BeamDeprecationWarning: BigQuerySource is deprecated since 2.25.0. Use ReadFromBigQuery instead.
  from ipykernel import kernelapp as app


/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:2527: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


AssertionError: Job did not reach to a terminal state after waiting indefinitely. Console URL: https://console.cloud.google.com/dataflow/jobs/<RegionId>/2022-07-11_07_27_47-12338818112686689332?project=<ProjectId>

In [ ]:
#!gsutil ls {DATA_DIR}/*
!ls {CANDIDATE_DIR}/*

# Version 2

### Reading from BigQuery

* To read an entire BigQuery table, use the table parameter with the BigQuery table name.

In [ ]:
PROJECT_ID = 'hybrid-vertex'
BUCKET_NAME = 'spotify-beam-v1' # 'spotify-tfrecords-blog' # Set your Bucket name
REGION = 'us-central1' # Set the region for Dataflow jobs
VERSION = 'v1'

ROOT = f'gs://{BUCKET_NAME}/{VERSION}'

CANDIDATE_DIR = ROOT + "/candidates/"
DATA_DIR = ROOT + '/data/' # Location to store data
STATS_DIR = ROOT +'/stats/' # Location to store stats 
STAGING_DIR = ROOT + '/job/staging/' # Dataflow staging directory on GCP
TEMP_DIR =  ROOT + '/job/temp/' # Dataflow temporary directory on GCP

### Define Table

In [66]:
PROJECT_ID = 'hybrid-vertex'
BQ_DATASET = 'spotify_train_3'
BQ_TABLE = 'train_flatten'

table_spec = f'{PROJECT_ID}:{BQ_DATASET}.{BQ_TABLE}'
output_path = DATA_DIR

total_samples = 65346428 
samples_per_file = 12228 
num_tfrecords = total_samples // samples_per_file

if num_tfrecords % total_samples:
    num_tfrecords += 1
    
print("Number of Expected TFRecords: {}".format(num_tfrecords)) # 5343

Number of Expected TFRecords: 5344


In [67]:
CONFIG = {
    "data": {
        "bq_source_table": table_spec,
        "output_path": output_path,
        'samples_per_file': samples_per_file,
        'num_tfrecords': num_tfrecords,
        "schema": [
            # playlist - context features
            { "name": "name", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "collaborative", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "n_songs_pl", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "num_artists_pl", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "num_albums_pl", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "description_pl", "kind": "tf.string", "mode": "NULLABLE" },
            # seed track - context features
            { "name": "track_name_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "artist_name_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "album_name_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "track_uri_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "artist_uri_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "album_uri_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "duration_seed_track", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "track_pop_seed_track", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "artist_pop_seed_track", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "artist_genres_seed_track", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "artist_followers_seed_track", "kind": "tf.float32", "mode": "NULLABLE" },
            # candidate - context features
            { "name": "track_name_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "artist_name_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "album_name_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "track_uri_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "artist_uri_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "album_uri_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "duration_ms_can", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "track_pop_can", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "artist_pop_can", "kind": "tf.float32", "mode": "NULLABLE" },
            { "name": "artist_genres_can", "kind": "tf.string", "mode": "NULLABLE" },
            { "name": "artist_followers_can", "kind": "tf.float32", "mode": "NULLABLE" },
            # Ragged Features
            { "name": "track_name_pl", "kind": "tf.string", "mode": "REPEATED" },
            { "name": "artist_name_pl", "kind": "tf.string", "mode": "REPEATED" },
            { "name": "album_name_pl", "kind": "tf.string", "mode": "REPEATED" },
            { "name": "track_uri_pl", "kind": "tf.string", "mode": "REPEATED" },
            { "name": "duration_ms_songs_pl", "kind": "tf.float32", "mode": "REPEATED" },
            { "name": "artist_pop_pl", "kind": "tf.float32", "mode": "REPEATED" },
            { "name": "artists_followers_pl", "kind": "tf.float32", "mode": "REPEATED" },
            { "name": "track_pop_pl", "kind": "tf.float32", "mode": "REPEATED" },
            { "name": "artist_genres_pl", "kind": "tf.string", "mode": "REPEATED" },
        ]
    },
    "tasks": [
        { "name": "analysis" },
        { "name": "training", "transform": "analysis", "shuffle": "True" },
        # { "name": "validation", "transform": "analysis" },
        # { "name": "testing", "transform": "identity" }
    ]
}

In [68]:
class TrainTfSeqExampleDoFn(beam.DoFn):
    """
    Convert training sample into TFExample
    """
    def __init__(self, task):
        """
        Initialization
        """
        self.task = task

    @staticmethod
    def _bytes_feature(value):
        """
        Get byte features
        """
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    @staticmethod
    def _int64_feature(value):
        """
        Get int64 feature
        """
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
    @staticmethod
    def _string_array(value):
        """
        Returns a bytes_list from a string / byte.
        """
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v.encode('utf-8') for v in value]))
    
    @staticmethod
    def _float_feature(value):
        """Returns a float_list from a float / double."""
        return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v) for v in value]))
    
    def process(self, data):
        """
        Convert BQ row to tf-example
        """
    
        # ===============================
        # Ragged Features - Query
        # ===============================
        ragged_key_list = [
            'track_name_pl',
            'artist_name_pl',
            'album_name_pl',
            # 'track_uri_pl',
            'duration_ms_songs_pl',
            'artist_pop_pl',
            'artists_followers_pl',
            'track_pop_pl',
            'artist_genres_pl',
        ]

        ragged_dict = {}

        for _ in ragged_key_list:
            ragged_dict[_] = []

        for x in data['track_name_pl']:
            ragged_dict['track_name_pl'].append(x.encode('utf8'))

        for x in data['artist_name_pl']:
            ragged_dict['artist_name_pl'].append(x.encode('utf8'))

        for x in data['album_name_pl']:
            ragged_dict['album_name_pl'].append(x.encode('utf8'))

        # for x in data['track_uri_pl']:
        #     ragged_dict['track_uri_pl'].append(x.encode('utf8'))

        for x in data['duration_ms_songs_pl']:
            ragged_dict['duration_ms_songs_pl'].append(x)

        for x in data['artist_pop_pl']:
            ragged_dict['artist_pop_pl'].append(x)

        for x in data['artists_followers_pl']:
            ragged_dict['artists_followers_pl'].append(x)

        for x in data['track_pop_pl']:
            ragged_dict['track_pop_pl'].append(x)

        for x in data['artist_genres_pl']:
            ragged_dict['artist_genres_pl'].append(x.encode('utf8'))

        # Set List Types
        # Bytes
        track_name_pl = BytesList(value=ragged_dict['track_name_pl'])
        artist_name_pl = BytesList(value=ragged_dict['artist_name_pl'])
        album_name_pl = BytesList(value=ragged_dict['album_name_pl'])
        # track_uri_pl = BytesList(value=ragged_dict['track_uri_pl'])
        artist_genres_pl = BytesList(value=ragged_dict['artist_genres_pl'])

        # Float List
        duration_ms_songs_pl = FloatList(value=ragged_dict['duration_ms_songs_pl'])
        artist_pop_pl = FloatList(value=ragged_dict['artist_pop_pl'])
        artists_followers_pl = FloatList(value=ragged_dict['artists_followers_pl'])
        track_pop_pl = FloatList(value=ragged_dict['track_pop_pl'])

        # Set FeatureLists
        # Bytes
        track_name_pl = FeatureList(feature=[Feature(bytes_list=track_name_pl)])
        artist_name_pl = FeatureList(feature=[Feature(bytes_list=artist_name_pl)])
        album_name_pl = FeatureList(feature=[Feature(bytes_list=album_name_pl)])
        # track_uri_pl = FeatureList(feature=[Feature(bytes_list=track_uri_pl)])
        artist_genres_pl = FeatureList(feature=[Feature(bytes_list=artist_genres_pl)])

        # Float Lists
        duration_ms_songs_pl = FeatureList(feature=[Feature(float_list=duration_ms_songs_pl)])
        artist_pop_pl = FeatureList(feature=[Feature(float_list=artist_pop_pl)])
        artists_followers_pl = FeatureList(feature=[Feature(float_list=artists_followers_pl)])
        track_pop_pl = FeatureList(feature=[Feature(float_list=track_pop_pl)])
        
        # ===============================
        # Create Context Features
        # ===============================
        context_features = {
            # playlist - context features
            "name": _string_array(data['name']),
            'collaborative' : _string_array(data['collaborative']),
            # 'duration_ms_seed_pl' : _float_feature(data['duration_ms_seed_pl']),
            'n_songs_pl' : _float_feature(data['n_songs_pl']),
            'num_artists_pl' : _float_feature(data['num_artists_pl']),
            'num_albums_pl' : _float_feature(data['num_albums_pl']),
            'description_pl' : _string_array(data['description_pl']),

            # seed track - context features
            'track_name_seed_track' : _string_array(data['track_name_seed_track']),
            'artist_name_seed_track' : _string_array(data['artist_name_seed_track']),
            'album_name_seed_track' : _string_array(data['album_name_seed_track']),
            # 'track_uri_seed_track' : _string_array(data['track_uri_seed_track']),
            # 'artist_uri_seed_track' : _string_array(data['artist_uri_seed_track']),
            # 'album_uri_seed_track' : _string_array(data['album_uri_seed_track']),
            'duration_seed_track' : _float_feature(data['duration_seed_track']),
            'track_pop_seed_track' : _float_feature(data['track_pop_seed_track']),
            'artist_pop_seed_track' : _float_feature(data['artist_pop_seed_track']),
            'artist_genres_seed_track' : _string_array(data['artist_genres_seed_track']),
            'artist_followers_seed_track' : _float_feature(data['artist_followers_seed_track']),

            #candidate features
            "track_name_can": _string_array(data['track_name_can']), 
            "artist_name_can": _string_array(data['artist_name_can']),
            "album_name_can": _string_array(data['album_name_can']),
            "track_uri_can": _string_array(data['track_uri_can']),
            # "artist_uri_can": _string_array(data['artist_uri_can']),
            # "album_uri_can": _string_array(data['album_uri_can']),
            "duration_ms_can": _float_feature(data['duration_ms_can']),
            "track_pop_can": _float_feature(data['track_pop_can']), 
            "artist_pop_can": _float_feature(data['artist_pop_can']),
            "artist_genres_can": _string_array(data['artist_genres_can']),
            "artist_followers_can": _float_feature(data['artist_followers_can']),
        }
        
        # ===============================
        # Create Sequence
        # ===============================
        seq = SequenceExample(
            context=tf.train.Features(
                feature=context_features
            ),
            feature_lists=FeatureLists(
                feature_list={
                    "track_name_pl": track_name_pl,
                    "artist_name_pl": artist_name_pl,
                    "album_name_pl": album_name_pl,
                    # "track_uri_pl": track_uri_pl,
                    "duration_ms_songs_pl": duration_ms_songs_pl,
                    "artist_pop_pl": artist_pop_pl,
                    "artists_followers_pl": artists_followers_pl,
                    "track_pop_pl": track_pop_pl,
                    "artist_genres_pl": artist_genres_pl
                }
            )
        )

        yield seq
        
# def _filter(mode, example):
#         return mode['name'] in example['mode']

### Python

In [69]:
PROJECT_ID = 'hybrid-vertex'
BUCKET_NAME = 'spotify-beam-v1' # 'spotify-tfrecords-blog' # Set your Bucket name
REGION = 'us-central1' # Set the region for Dataflow jobs
VERSION = 'v1'

ROOT = f'gs://{BUCKET_NAME}/{VERSION}'

CANDIDATE_DIR = ROOT + "/candidates/"
DATA_DIR = ROOT + '/data/' # Location to store data
STATS_DIR = ROOT +'/stats/' # Location to store stats 
STAGING_DIR = ROOT + '/job/staging/' # Dataflow staging directory on GCP
TEMP_DIR =  ROOT + '/job/temp/' # Dataflow temporary directory on GCP

MAX_WORKERS = '20'

RUNNER = 'DataflowRunner'
STAGING_LOCATION = STAGING_DIR
TEMP_LOCATION = TEMP_DIR

TIMESTAMP = datetime.utcnow().strftime('%y%m%d-%H%M%S')
JOB_NAME = f'spotify-tfrecords-{VERSION}-{TIMESTAMP}'


In [70]:
# import sys

# sys.path

# spotify_mpd_two_tower/jt-dev/pipeline/schema.py

In [71]:
from schema import Schema

# def pipeline():

"""
Main function to create tfrecords (ETL pipeline)
"""
RUNNER = 'DataflowRunner'

# Read the SQL code
table = CONFIG['data']['bq_source_table'] #.read()
schema = CONFIG['data']['schema']
output_path = CONFIG['data']['output_path']
num_tfrecords = CONFIG['data']['num_tfrecords']

# --> EXTRACTION <--

# Create a BigQuery source
source = beam.io.gcp.bigquery.ReadFromBigQuery(table=table, use_standard_sql=True, flatten_results=False)

# Transform to Sequence Example
create_sequence_examples = TrainTfSeqExampleDoFn(task = 'train')

# --> LOADING <--
# Write serialized example to tfrecords
write_to_tf_record = beam.io.tfrecordio.WriteToTFRecord(
    file_path_prefix=output_path,
    file_name_suffix='.tfrecord',
    num_shards=num_tfrecords)

# TODO: Enter required pipeline arguments (Method 1)
pipeline_args = [
    '--runner', RUNNER,
    '--project', PROJECT_ID,
    '--region', REGION,
    '--staging_location', STAGING_DIR,
    '--temp_location', TEMP_DIR,
    # '--template_location', TEMPLATE_LOCATION,
    '--job_name', JOB_NAME,
    '--num_workers', MAX_WORKERS,
    # '--setup_file', './setup.py'
]
pipeline_options = PipelineOptions(pipeline_args)
pipeline_options.view_as(SetupOptions).save_main_session = True

# Defining Apache Beam Pipeline
with beam.Pipeline(options=pipeline_options) as pipe:
    _ = (pipe
         | 'ReadBQTable' >> source
         | 'TrainToSequenceExample' >> beam.ParDo(create_sequence_examples)
         | 'SerializeProto' >> beam.Map(lambda x: x.SerializeToString())
         | 'WriteTfrecord' >> write_to_tf_record)

AssertionError: Job did not reach to a terminal state after waiting indefinitely. Console URL: https://console.cloud.google.com/dataflow/jobs/<RegionId>/2022-07-11_15_15_45-16356143904307732923?project=<ProjectId>

# Scratch

In [45]:
from spotify_mpd_two_tower import Schema

# Read the SQL code
table = CONFIG['data']['bq_source_table'] #.read()
schema = CONFIG['data']['schema']
output_path = CONFIG['data']['output_path']

# Create a BigQuery source
source = beam.io.gcp.bigquery.ReadFromBigQuery(table=table, use_standard_sql=True, flatten_results=False)

# Create metadata needed later
spec = schema.to_feature_spec()
meta = dataset_metadata.DatasetMetadata(schema=dataset_schema.from_feature_spec(spec))

# Read from BigQuery
data = pipeline \
    | 'read' >> beam.io.Read(source)

# Loop over tasks whose purpose is analysis
transform_functions = {}
for task in config['tasks']:
    if 'transform' in task:
        continue
    name = task['name']
    
transform_functions[name] \
    | name + '-write-transform' >> transform_fn_io.WriteTransformFn(output_path)

# Loop over tasks whose purpose is transformation
for task in config['tasks']:
    if not 'transform' in task:
        continue
    name = task['name']
    # Select examples that belong to the current mode
    data_ = data \
        | name + '-filter' >> beam.Filter(partial(_filter, task))
    # Shuffle examples if needed
    if task.get('shuffle', False):
        data_ = data_ \
            | name + '-shuffle' >> beam.transforms.Reshuffle()
        
        # Transform the examples using an appropriate transform function
        if task['transform'] == 'identity':
            coder = tft.coders.ExampleProtoCoder(meta.schema)
        else:
            data_, meta_ = ((data_, meta), transform_functions[task['transform']]) \
                | name + '-transform' >> tt_beam.TransformDataset()
            coder = tft.coders.ExampleProtoCoder(meta_.schema)
            
            
        # Store the transformed examples as TFRecords
        data_ \
            | name + '-encode' >> beam.Map(coder.encode) \
            | name + '-write-examples' >> beam.io.tfrecordio.WriteToTFRecord(path)

<ReadFromBigQuery(PTransform) label=[ReadFromBigQuery] at 0x7f2aecbdcf10>

In [ ]:
# Read the SQL code
table = open(config['data']['bq_source_table']).read()

# Create a BigQuery source
source = beam.io.BigQuerySource(table=table, use_standard_sql=True, flatten_results=False)

# Create metadata needed later
spec = schema.to_feature_spec()
meta = dataset_metadata.DatasetMetadata(
    schema=dataset_schema.from_feature_spec(spec)
)

data = pipeline \
    | 'read' >> beam.io.Read(source)